In [2]:
!pip install keras

Looking in indexes: http://mirrors.tencentyun.com/pypi/simple
     |████████████████████████████████| 377 kB 818 kB/s eta 0:00:01


In [21]:
from keras.layers import Dense, Input, LSTM, Embedding, BatchNormalization,Bidirectional,GlobalMaxPool1D
import os
import gc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import pandas as pd
import numpy as np
import keras
import h5py
from keras.optimizers import RMSprop


def buid_model():
    # LSTM 模型
    print('lstm model start...\n')

    # 标题输入：接收一个含有 200 个整数的序列，每个整数在 1 到 3812202 之间。
    # 标题输入：接收一个含有 200 个整数的序列，每个整数在 1 到 3812202 之间。
    main_input1 = Input(shape=(200,), name='main_input1', dtype='int32')
    with h5py.File('../../get_w2v_feat/w2v1_pre_ad/embeddings_matrix_ad_size_32_w200_count_0.h5','r') as f:
        embeddings_matrix1 = np.array(f.get('embeddings_matrix'))
    embeddings1 = Embedding(input_dim = len(embeddings_matrix1), # 字典长度
                                output_dim = 32, # 词向量 长度（100）
                                weights=[embeddings_matrix1], # 重点：预训练的词向量系数
                                input_length=200, # 每句话的 最大长度（必须padding） 
                                trainable= True, # 是否在 训练的过程中 更新词向量
                                mask_zero = True)(main_input1)
    
    main_input2 = Input(shape=(200,), name='main_input2', dtype='int32')
    with h5py.File('../../get_w2v_feat/w2v1_pre_advertiser/embeddings_matrix_advertiser_size_32_w200_count_5.h5','r') as f:
        embeddings_matrix2 = np.array(f.get('embeddings_matrix'))
    embeddings2 = Embedding(input_dim = len(embeddings_matrix2), # 字典长度
                                output_dim = 32, # 词向量 长度（100）
                                weights=[embeddings_matrix2], # 重点：预训练的词向量系数
                                input_length=200, # 每句话的 最大长度（必须padding） 
                                trainable= True, # 是否在 训练的过程中 更新词向量
                                mask_zero = True)(main_input2)
    
    main_input3 = Input(shape=(200,), name='main_input3', dtype='int32')
    with h5py.File('../../get_w2v_feat/w2v1_pre_creative_id/embeddings_matrix_creative_size_32_w200_count_5.h5','r') as f:
        embeddings_matrix3 = np.array(f.get('embeddings_matrix'))
    embeddings3 = Embedding(input_dim = len(embeddings_matrix3), # 字典长度
                                output_dim = 32, # 词向量 长度（100）
                                weights=[embeddings_matrix3], # 重点：预训练的词向量系数
                                input_length=200, # 每句话的 最大长度（必须padding） 
                                trainable= True, # 是否在 训练的过程中 更新词向量
                                mask_zero = True)(main_input3)
    
    main_input4 = Input(shape=(200,),name='main_input4',dtype='int32')
    with h5py.File('../../get_w2v_feat/w2v1_pre_industry/embeddings_matrix_industry_size_32_w200_count_5.h5','r') as f:
        embeddings_matrix4 = np.array(f.get('embeddings_matrix'))
    embeddings4 = Embedding(input_dim = len(embeddings_matrix4), # 字典长度
                                output_dim = 32, # 词向量 长度（32）
                                weights=[embeddings_matrix4], # 重点：预训练的词向量系数
                                input_length=200, # 每句话的 最大长度（必须padding） 
                                trainable=True, # 是否在 训练的过程中 更新词向量
                                mask_zero = False)(main_input4)
    main_input5 = Input(shape=(200,),name='main_input5',dtype='int32')
    with h5py.File('../../get_w2v_feat/w2v1_pre_product_id/embeddings_matrix_product_id_size_32_w200_count_5.h5','r') as f:
        embeddings_matrix5 = np.array(f.get('embeddings_matrix'))
    embeddings5 = Embedding(input_dim = len(embeddings_matrix5), # 字典长度
                                output_dim = 32, # 词向量 长度（32）
                                weights=[embeddings_matrix5], # 重点：预训练的词向量系数
                                input_length=200, # 每句话的 最大长度（必须padding） 
                                trainable=True, # 是否在 训练的过程中 更新词向量
                                mask_zero = False)(main_input5)
    
    emb = keras.layers.concatenate([embeddings1, embeddings2, embeddings3,
                                    embeddings4, embeddings5])
    
    lstm_out = Bidirectional(LSTM(64,activation='softsign',return_sequences=False))(emb)
#     lstm_out = Bidirectional(LSTM(100,activation='softsign',return_sequences=False))(lstm_out)
    
#     lstm_out = GlobalMaxPool1D()(lstm_out)
    # 堆叠全连接
#     den_x = Dense(32, activation='relu')(lstm_out)
    
#     # 辅助输入数据
#     auxiliary_input = Input(shape=(48,), name='aux_input')
#     # 将辅助输入与lstm输出连接起来
    
#     lstm_out = Dense(30, activation='relu')(lstm_out)
#     lstm_out = BatchNormalization()(lstm_out)
    # 辅助损失函数
#     auxiliary_output = Dense(1, activation='sigmoid', name='aux_output')(lstm_out)
    # 主输出
    main_output = Dense(1, activation='sigmoid', name='main_output')(lstm_out)
    

    # 定义一个具有两个输入输出的模型
    model = keras.models.Model(inputs=[main_input1,main_input2,main_input3,
                                      main_input4,main_input5],
                               outputs=[main_output])  # 这里的输入输出顺序与fit时一致就好
    
    opt = RMSprop(lr=0.005,  clipnorm=1.0)
    model.compile(optimizer=opt,
                  loss={'main_output': 'binary_crossentropy'},
                 metrics=['accuracy'])

    print('lstm model geted...\n')
    return model

def data_load():
    print('loading data ... \n')

    with h5py.File('../../get_w2v_feat/w2v1_pre_ad/word_train_ad_w2v_w200.h5', 'r') as f:
        data = np.array(f.get('word_data'))
        
    label = pd.read_csv('../../train_preliminary/user.csv').sort_values(by=['user_id'])

    train_x, test_x, train_y, test_y = train_test_split(data, label, test_size=0.2, random_state=2020)

    train_y_age = train_y['age'].values - 1
    train_y_age = keras.utils.np_utils.to_categorical(train_y_age, num_classes=10)
    train_y_gender = train_y['gender'].values - 1

    test_y_age = test_y['age'].values - 1
    test_y_age = keras.utils.np_utils.to_categorical(test_y_age, num_classes=10)
    test_y_gender = test_y['gender'].values - 1

    print('get data ... \n')

    return train_x, test_x, train_y_age, train_y_gender,test_y_age,test_y_gender

def load_data2():
    
    with h5py.File('../../get_w2v_feat/w2v1_pre_advertiser/word_train_advertiser_w2v_w200.h5', 'r') as f:
        data = np.array(f.get('word_data'))

    train_x, test_x= train_test_split(data, test_size=0.2, random_state=2020)
    return train_x, test_x

def load_data3():
    
    with h5py.File('../../get_w2v_feat/w2v1_pre_creative_id/word_train_creative_w2v_w200.h5', 'r') as f:
        data = np.array(f.get('word_data'))

    train_x, test_x= train_test_split(data, test_size=0.2, random_state=2020)
    return train_x, test_x
def load_data4():
    
    print('loading data1 ... \n')
    with h5py.File('../../get_w2v_feat/w2v1_pre_industry/word_train_industry_w2v_w200.h5', 'r') as f:
        data = np.array(f.get('word_data'))
    train_x, test_x= train_test_split(data, test_size=0.2, random_state=2020)
    print('get data2 ... \n')
    return train_x, test_x

def load_data5():
    
    print('loading data1 ... \n')
    with h5py.File('../../get_w2v_feat/w2v1_pre_product_id/word_train_product_id_w2v_w200.h5', 'r') as f:
        data = np.array(f.get('word_data'))
    train_x, test_x= train_test_split(data, test_size=0.2, random_state=2020)
    print('get data2 ... \n')
    return train_x, test_x

In [22]:

def get_filename_for_saving(save_dir):
    return os.path.join(save_dir,
                        "lstm32_comb_gender_w2v_200_pre_5in_{val_loss:.3f}-{val_acc:.3f}-{epoch:03d}-{loss:.3f}-{acc:.3f}.hdf5")


model = buid_model()

lstm model start...

lstm model geted...



In [23]:
train_x, test_x, train_y_age, train_y_gender,test_y_age,test_y_gender = data_load()

train_x2, test_x2 = load_data2()

train_x3, test_x3 = load_data3()

train_x4, test_x4 = load_data4()
train_x5, test_x5 = load_data5()

loading data ... 

get data ... 

loading data1 ... 

get data2 ... 

loading data1 ... 

get data2 ... 



In [24]:
print('lstm model fit...\n')
checkpointer = keras.callbacks.ModelCheckpoint(
    filepath=get_filename_for_saving(''),
    save_best_only=False)
stopping = keras.callbacks.EarlyStopping(patience=8)
reduce_lr = keras.callbacks.ReduceLROnPlateau(factor=0.1, patience=2, min_lr=0.0001)

model.fit({'main_input1': train_x ,'main_input2': train_x2,'main_input3': train_x3,
          'main_input4': train_x4,'main_input5': train_x5},
          {'main_output': train_y_gender},
          epochs=1,
          batch_size=256,
          validation_data=({'main_input1': test_x,'main_input2': test_x2,'main_input3': test_x3,
                           'main_input4': test_x4,'main_input5': test_x5},
                           {'main_output': test_y_gender}),
          callbacks=[checkpointer, reduce_lr, stopping])

lstm model fit...

Train on 720000 samples, validate on 180000 samples
Epoch 1/1
720000/720000 [==============================] - 4637s 6ms/step - loss: 0.1779 - acc: 0.9359 - val_loss: 0.1663 - val_acc: 0.9416
